## This file reads the review data and clean it, join with user data as well
Review data combined user data and business data 

raw review data: 5996996

review cleaned: 3390939

In [27]:
import os 
import json 
import tqdm 

from ast import literal_eval
from os import listdir
from os.path import isfile, join
from scipy.sparse import csr_matrix, load_npz, save_npz

import datetime
import json
import numpy as np
import pandas as pd
import time
import yaml
import sys

In [2]:
# file directory
file_business, file_review, file_user = [
    '..\\data\\business.json',
    '..\\data\\yelp_academic_dataset_review.json',
    '..\\data\\user.json',
]

### Read in review data 
df has 5996996 raw review histories

df_join has 3392380 review histories after selecting the ones that has user average star history

In [3]:
#path = "yelp_academic_dataset_review.json"

#Litos' way
#file = open(path, encoding="utf8")
#data = file.readlines()
#data = list(map(json.loads, data))

#Json's way
with open(file_review, encoding = 'utf-8') as json_file:
    data = json_file.readlines()
    data = list(map(json.loads, data))

df_review = pd.DataFrame(data)

In [4]:
df_review.head(2)  #5996995   review histories

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,iCQpiavjjPzJ5_3gPD5Ebg,0,2011-02-25,0,x7mDIiDB3jEiPGPHOmDzyw,2,The pizza was okay. Not the best I've had. I p...,0,msQe1u7Z_XuqjGoqhB0J5g
1,pomGBqfbxcqPv14c3XH-ZQ,0,2012-11-13,0,dDl8zu1vWPdKGihJrwQbpw,5,I love this place! My fiance And I go here atl...,0,msQe1u7Z_XuqjGoqhB0J5g


In [82]:
df_review.shape[0]

5996996

### Reading in user data 
total users: 1637138 

In [7]:
# loading review data to pandas
with open(file_user, encoding = 'utf-8') as f:
    df_user = pd.DataFrame(json.loads(line) for line in f)

#no user that has no average rating history
#df_user.average_stars.isna().sum()
#df_user.average_stars.min()
#df_user.loc[df_user.average_stars == '1.0']

In [8]:
df_user.head(2)

,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,...,cool,elite,fans,friends,funny,name,review_count,useful,user_id,yelping_since
0,4.03,1,0,1,2,0,0,1,0,1,...,25,"2015,2016,2017",5,"c78V-rj8NQcQjOI8KP3UEA, alRMgPcngYSCJ5naFRBz5g...",17,Rashmi,95,84,l6BmjZMeQD3rDxWUbiAiow,2013-10-08 23:11:33
1,3.63,1,0,1,1,0,0,0,0,0,...,16,,4,"kEBTgDvFX754S68FllfCaA, aB2DynOxNOJK9st2ZeGTPg...",22,Jenna,33,48,4XChL029mKr5hydo79Ljxg,2013-02-21 22:29:06


In [9]:
#Select the information we want from the users 
selected_features = ['user_id','average_stars']
df_user_selectedStars = df_user.filter(items = selected_features)
df_user_selectedStars.head(5)

,user_id,average_stars
0,l6BmjZMeQD3rDxWUbiAiow,4.03
1,4XChL029mKr5hydo79Ljxg,3.63
2,bc8C_eETBWL0olvFSJJd0w,3.71
3,dD0gZpBctWGdWo9WlGuhlA,4.85
4,MM4RJAeH6yuaN8oZDSt0RA,4.08


### Combining review data and user average rating data, inner join on user_id

In [10]:
# set user_id as the index of two data frames
df_user_selectedStars = df_user_selectedStars.set_index('user_id')
df_review = df_review.set_index('user_id')

In [11]:
df_review.head(2)

,business_id,cool,date,funny,review_id,stars,text,useful
user_id,,,,,,,,
msQe1u7Z_XuqjGoqhB0J5g,iCQpiavjjPzJ5_3gPD5Ebg,0,2011-02-25,0,x7mDIiDB3jEiPGPHOmDzyw,2,The pizza was okay. Not the best I've had. I p...,0
msQe1u7Z_XuqjGoqhB0J5g,pomGBqfbxcqPv14c3XH-ZQ,0,2012-11-13,0,dDl8zu1vWPdKGihJrwQbpw,5,I love this place! My fiance And I go here atl...,0


In [12]:
df_user_selectedStars.head(2)

,average_stars
user_id,
l6BmjZMeQD3rDxWUbiAiow,4.03
4XChL029mKr5hydo79Ljxg,3.63


In [13]:
# combine two dataset with the same user id 

df_join = df_review.join(df_user_selectedStars,how = 'inner')
df_join.reset_index(inplace = True)

In [14]:
# rename the column name "stars" to "avg_stars" in business dataset to aviod nameing conflicts with review dataset
df_join = df_join.rename(index = str, columns = {"average_stars":"user_avg_stars"})

In [15]:
#df_join.to_json('reviewNuserAvgFile.json',orient='records', lines=True)
#data_hasUserAvg = df_join.to_json(orient='records', lines=True)
df_join.size

59032150

In [16]:
#df_join.to_json(r'reviewWuserAvg.json',orient='split')

### Read in business data
192608 raw business

cutting 30000 business that are closed 

cutting 20000 business where attributes and categories are null

cutting 80000 non-resaurant businesses

52638 valid businesses for use 

In [17]:
# loading business data to pandas
with open(file_business, encoding = 'utf-8') as f:
    df_business = pd.DataFrame(json.loads(line) for line in f)

In [29]:
np.set_printoptions(threshold=sys.maxsize)
df_business.state.unique()

array(['AZ', 'ON', 'NC', 'AB', 'NV', 'OH', 'PA', 'QC', 'WI', 'IL', 'NY',
       'SC', 'TX', 'UT', 'NM', 'FL', 'CA', 'VA', 'BAS', 'NE', 'AK', 'XGM',
       'WA', 'XWY', 'CON', 'BC', 'GA', 'VT', 'CT', 'AL', 'DUR', 'TN',
       'NJ', 'AR', 'XGL', 'DOW'], dtype=object)

In [73]:
#Selecting only the ON businesses 
df_ON_business = df_business[df_business['state'] == 'ON']

In [74]:
df_business.shape[0]

192609

In [75]:
df_ON_business.shape[0]

33412

In [34]:
(df_ON_business['is_open'] == 1).sum()

26525

In [20]:
# remove closed business
#df_business = df_business[df_business['is_open'] == 1]

In [36]:
print('Number of business that are opened',(df_ON_business['is_open'] == 1).sum())

Number of business that are opened 26525


#### Clean the business data
filter out the businesses that are closed, and not restaurants, and probably that don't have attributes/categories

In [27]:
#Only select certain features to join 
selected_features_for_join = ['business_id','categories', 'stars']
df_business_forJoin = df_business.filter(items = selected_features_for_join)
df_business_forJoin = df_business_forJoin.rename(index = str, columns = {"stars": "business_stars"})

#### Selecting only the ON businesses

In [76]:
#Select ON businesses
selected_features_for_join = ['business_id','categories', 'stars', 'is_open','address', 'city', 'hours', 'latitude', 'longitude',\
                             'name', 'review_count', 'stars']
df_ON_business_forJoin = df_ON_business.filter(items = selected_features_for_join)
df_ON_business_forJoin = df_ON_business_forJoin.rename(index = str, columns = {"stars": "business_stars"})

In [28]:
# selected_features_for_join = ['name','business_id','categories']
# df_business_forJoin = df_business.filter(items = selected_features_for_join)

In [29]:
#pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_columns', None)  
# df_business_forJoin.categories


In [30]:
# remove null row for categories column and attributes column
cond_category_not_null = ~df_business_forJoin['categories'].isnull()
# cond_attribute_not_null = ~df_business_forJoin['attributes'].isnull()
df_business_forJoin = df_business_forJoin[cond_category_not_null]
print('Number of business that are opened, and categories & attributes not null', df_business_forJoin.shape[0])

Number of business that are opened, and categories & attributes not null 192127


In [77]:
#Filtering for toronto businesses 
cond_ON_category_not_null = ~df_ON_business_forJoin['categories'].isnull()
# cond_attribute_not_null = ~df_business_forJoin['attributes'].isnull()
df_ON_business_forJoin = df_ON_business_forJoin[cond_ON_category_not_null]
print('Number of TORONTO business that categories not null', df_ON_business_forJoin.shape[0])

Number of TORONTO business that categories not null 33342


In [31]:
# attributeSet = []
# for attribute in df_business_forJoin['attributes']:
#     for (key, item) in attribute.items():
#         attributeSet.append(key)
# arr = np.array(attributeSet)
# np.unique(arr)

# cateSet = []
# for attribute in df_business_forJoin['categories']:
#         attributeSet.append(attribute)

# arr = np.array(cateSet)
# np.unique(arr)

###### Get the list of attribute and the list of categories 

In [32]:
# listAttribute = []
# for attributes in df_business_forJoin[~df_business_forJoin["categories"].str.contains("Restaurants|restaurant|Bakeries|Food|Coffee|Tea|Desserts|food|Cupcakes",case=False)].attributes:
#     for key in attributes.keys():
#         listAttribute.append(key)
        
# from collections import Counter

# c = Counter(listAttribute)

# listCategory = []
# for category in df_business_forJoin[~df_business_forJoin["categories"].str.contains("Restaurants|restaurant|Bakeries|Food|Coffee|Tea|Desserts|food|Cupcakes|Bar|Nightlife|Pub|Lounges|Beer|Breakfast",case=False)].categories:
#     splitWords = [words.strip(",.&") for words in category.split()]
#     for word in splitWords:
#         if word != '':
#             listCategory.append(word)
            
# from collections import Counter

# count = Counter(listCategory)
# #count

In [33]:
#count

In [34]:
# We only take restaurant business
cond_category_restaurant = df_business_forJoin["categories"].str.contains("Restaurants|restaurants|Bakeries|Foods|Coffee|Tea|Desserts|food|Cupcakes|cafes|Breakfast|Lunch|Dinner",case=False)
#cannot use attribute over here
#cond_attribute_restaurant = df_business_forJoin["attributes"].str.contains("Restaurants|restaurant",case=False)
df_business_forJoin = df_business_forJoin[cond_category_restaurant]

In [35]:
print('number of businesses that are opened restaurants, and categories & attributes not null',df_business_forJoin.shape[0])

number of businesses that are opened restaurants, and categories & attributes not null 75467


#### Counting number of restaurants in ON

In [78]:
#Counting restaurant businesses
cond_category_restaurant = df_ON_business_forJoin["categories"].str.contains("Restaurants|restaurants|Bakeries|Foods|Coffee|Tea|Desserts|food|Cupcakes|cafes|Breakfast|Lunch|Dinner",case=False)
#cannot use attribute over here
#cond_attribute_restaurant = df_business_forJoin["attributes"].str.contains("Restaurants|restaurant",case=False)
df_ON_business_forJoin = df_ON_business_forJoin[cond_category_restaurant]

In [81]:
print('number of restaurants, and categories not null',df_ON_business_forJoin.shape[0])
print('number of restaurants that are opened and categories opened: ', (df_ON_business_forJoin['is_open'] == 1).sum())

number of restaurants, and categories not null 18518
number of restaurants that are opened and categories opened:  13436


number of restaurants, and categories not null 18518

number of restaurants that are opened and categories not null:  13436

In [40]:
#Now combine the restaurants with df_join on business id 
# set business_id as the index of two data frames
df_business_forJoin = df_business_forJoin.set_index('business_id')
df_join = df_join.set_index('business_id')

### Ready to combine business data with review data

In [80]:
#Now combine the restaurants with df_join on business id 
# set business_id as the index of two data frames
df_ON_business_forJoin = df_ON_business_forJoin.set_index('business_id')
df_join = df_join.set_index('business_id')

KeyError: 'business_id'

### Combine review dataset with business dataset on business id 

In [70]:
# combine two dataset with the same business id 
df_join_final = df_join.join(df_ON_business_forJoin,how = 'inner')
#Resetting the index
df_join_final.reset_index(inplace = True)
df_ON_business_forJoin.reset_index(inplace = True)

In [71]:
df_join_final.shape[0]

568159

In [85]:
np.set_printoptions(threshold=sys.maxsize)
listCity = []
for city in df_join_final.city:
    if city not in listCity:
        listCity.append(city)

In [86]:
listCity

['Richmond Hill',
 'Toronto',
 'East York',
 'North York',
 'Markham',
 'Newmarket',
 'Etobicoke',
 'Aurora',
 'Woodbridge',
 'Mississauga',
 'Ajax',
 'King City',
 'Scarborough',
 'Brampton',
 'Caledon East',
 'Pickering',
 'Gormley',
 'Whitby',
 'Vaughan',
 'Oakville',
 'York',
 'Bolton',
 'Bradford',
 'Whitchurch-Stouffville',
 'Stouffville',
 'Concord',
 'Thornhill',
 'Brooklin',
 'Beeton',
 'Rexdale',
 'King',
 'Etobiicoke',
 'Willowdale',
 'Mount Albert',
 'Georgetown',
 'Unionville',
 'Kleinburg',
 'Maple',
 'Caledon',
 'Schomberg',
 'East Gwillimbury',
 'Nobleton',
 'Clarkson',
 'Inglewood',
 'Bradford West Gwillimbury',
 'East Mississauga',
 'Sharon',
 'Downtown Toronto',
 'North of Brampton',
 'Palgrave',
 'North  York',
 'Oakridges',
 'Tornto',
 'Streetsville',
 'York Regional Municipality',
 'Thornhil',
 'AGINCOURT',
 'Missisauga',
 'THORNHILL',
 'ETOBICOKE',
 'Port Credit',
 'Whtiby',
 'Don Mills',
 'Mississuaga',
 'Agincourt',
 'Bond Head',
 'Woodbridge (Vaughan)',
 'East

In [41]:
# combine two dataset with the same business id 
df_join_final = df_join.join(df_business_forJoin,how = 'inner')
df_join_final.reset_index(inplace = True)
df_business_forJoin.reset_index(inplace = True)

In [42]:
df_join_final.shape[0]

4047190

In [43]:
#df_join_final = df_join_final.drop(columns = ['categories', 'attributes','business_stars'])
df_join_final.head(2)

,business_id,user_id,cool,date,funny,review_id,stars,text,useful,user_avg_stars,categories,business_stars
0,--1UhMGODdWsrMastO9DZw,1b4rlFFdNfeysOjbrFykng,0,2016-08-16,0,8UsalYi1Jkbk8WitFEa_aQ,4,Spicy Amigos is the old Jalapeños. I'm not sur...,0,3.14,"Restaurants, Mexican",4.0
1,--1UhMGODdWsrMastO9DZw,4YHZzwlxEMi7zWO6Osszdw,0,2017-07-26,0,X4PwY0mMHxqOf2O1pwYdIQ,5,"Delicious! One of the best burritos, salsa and...",0,3.92,"Restaurants, Mexican",4.0


### Export new cleaned data

In [44]:
Export = df_join_final.to_json(r'..\\data\\Export_CleanedReviewNew_1011.json', orient='records')

In [38]:
df_join_final.rename(columns={'stars': 'review_stars', 'text': 'review_text', 'cool': 'review_cool',
                       'funny': 'review_funny', 'useful': 'review_useful'},
              inplace=True)

In [42]:
import datetime
import time
def date_to_timestamp(date):
    dt = datetime.datetime.strptime(date, '%Y-%m-%d')
    return time.mktime(dt.timetuple())

In [43]:
df_join_final['business_num_id'] = df_join_final.business_id.astype('category').\
        cat.rename_categories(range(0, df_join_final.business_id.nunique()))
df_join_final['business_num_id'] = df_join_final['business_num_id'].astype('int')

df_join_final['user_num_id'] = df_join_final.user_id.astype('category').\
    cat.rename_categories(range(0, df_join_final.user_id.nunique()))
df_join_final['user_num_id'] = df_join_final['user_num_id'].astype('int')

df_join_final['timestamp'] = df_join_final['date'].apply(date_to_timestamp)

In [44]:
df_join_final['reviewStars-userAvg'] = df_join_final['review_stars'] - df_join_final['user_avg_stars']

In [63]:
df_join_final.head(2)

,business_id,user_id,review_cool,date,review_funny,review_id,review_stars,review_text,review_useful,user_avg_stars,business_num_id,user_num_id,timestamp,reviewStars-userAvg
0,--1UhMGODdWsrMastO9DZw,1b4rlFFdNfeysOjbrFykng,0,2016-08-16,0,8UsalYi1Jkbk8WitFEa_aQ,4,Spicy Amigos is the old Jalapeños. I'm not sur...,0,3.14,0,39816,1.471320e+09,0.86
1,--1UhMGODdWsrMastO9DZw,4YHZzwlxEMi7zWO6Osszdw,0,2017-07-26,0,X4PwY0mMHxqOf2O1pwYdIQ,5,"Delicious! One of the best burritos, salsa and...",0,3.92,0,85109,1.501042e+09,1.08


## Disregard this section 

### Matching the business table's business_num_id with review table's business_num_id

In [56]:
df_business_forJoin.loc[df_business_forJoin['business_id'].isin(df_join_final['business_id']), ['business_num_id']] = df_join_final[['business_num_id']]

In [79]:
for businessId in df_business_forJoin['business_id']:
    #df_business_forJoin[df_business_forJoin['business_id'] == businessId].business_num_id = df_join_final[df_join_final['business_id'] == businessId].business_num_id[0]
    tempBusNumId = df_join_final[df_join_final['business_id'] == businessId].business_num_id.iloc[0]
    df_business_forJoin[df_business_forJoin['business_id'] == businessId].business_num_id = tempBusNumId

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


IndexError: single positional indexer is out-of-bounds

'8UsalYi1Jkbk8WitFEa_aQ'

In [62]:
df_business_forJoin[df_business_forJoin.business_id =='--1UhMGODdWsrMastO9DZw']

,business_id,business_stars,categories,attributes,business_num_id
41190,--1UhMGODdWsrMastO9DZw,4.0,"Restaurants, Mexican","{'BusinessParking': '{'garage': False, 'street...",499


In [65]:
df_business_forJoin.head(10)

,business_id,business_stars,categories,attributes,business_num_id
0,QXAEGFB4oINsVuTFxEYKFQ,2.5,"Specialty Food, Restaurants, Dim Sum, Imported...","{'RestaurantsReservations': 'True', 'GoodForMe...",0
1,gnKjwL_1w79qoiV3IC_xQQ,4.0,"Sushi Bars, Restaurants, Japanese","{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...",0
2,1Dfx3zM-rW4n-31KeC8sJg,3.0,"Restaurants, Breakfast & Brunch, Mexican, Taco...","{'RestaurantsTakeOut': 'True', 'BusinessParkin...",0
3,fweCYi8FmbJXHCqLnwuk8w,4.0,"Italian, Restaurants, Pizza, Chicken Wings","{'RestaurantsPriceRange2': '2', 'BusinessAccep...",0
4,-K4gAv8_vjx8-2BxkVeRkA,3.0,"Bakeries, Food","{'RestaurantsTakeOut': 'True', 'BusinessParkin...",0
5,1RHY4K3BD22FK7Cfftn8Mg,4.0,"Sandwiches, Salad, Restaurants, Burgers, Comfo...","{'RestaurantsTakeOut': 'True', 'BusinessParkin...",0
6,tstimHoMcYbkSC4eBA1wEg,4.5,"Mexican, Restaurants, Patisserie/Cake Shop, Fo...","{'BikeParking': 'True', 'BusinessParking': '{'...",0
7,C9oCPomVP0mtKa8z99E3gg,4.5,"Bakeries, Food","{'RestaurantsDelivery': 'False', 'RestaurantsP...",0
8,NDuUMJfrWk52RA-H-OtrpA,3.0,"Juice Bars & Smoothies, Food, Restaurants, Fas...","{'WiFi': 'u'no'', 'BikeParking': 'True', 'Rest...",0
9,NBn4hgfGtNz91k3VsDZlmw,5.0,"Local Services, Food Banks, Community Service/...","{'RestaurantsPriceRange2': '1', 'BusinessAccep...",0


### Export the cleaned review data, with user user average rating and valid business locations

In [36]:
#Export = df_join_final.to_json(r'..\\data\\Export_CleanedReview.json', orient='records')

### The way to split the data into smaller portion

In [37]:
# total_len = len(data)
# num = 10
# chunk_len = total_len // num

# folder = "data"
# if not os.path.exists(folder):
#     os.makedirs(folder)
# for i in range(num):
#     name = os.path.join(folder, "%d.txt" % i) 
#     with open(name, 'a') as f:
#         for line in tqdm.tqdm(data[i*chunk_len :  (i+1)*chunk_len]):
#             data_s = json.dumps(line)+"\n"
#             f.write(data_s)
#     print("fininshed file: ", i)

# name = os.path.join(folder, "%d.txt" % num) 
# with open(name, 'a') as f:
#     for line in tqdm.tqdm(data[num*chunk_len :  total_len]):
#         data_s = json.dumps(line)+"\n"
#         f.write(data_s)
# print("fininshed file: ", i)